# Phase Transition Final Project

Kawasaki Dynamics

- Ising model have fixed spins. Don't change during simulation, but allowed to move (as in swap places)
- Movement Rules
1) For particles, A and B, if spins are the same, do nothing
2) if spins A and B are different, then calculate current system energy (E) and system energy if places were swapped (E')
3) Calculate probabilities according to Boltzmann distributions, p and p' respectively
4) With probability, p'/ p + p', if it hits, then swap the particles. 


Goal is to decrease the energy for every step


Link: https://rf.mokslasplius.lt/kawasaki-ising-model/

JS link: https://github.com/physrisk/website-source/blob/main/uploads/models/physics-models/ising-model/kawasaki.js

# 
Farkas Paper Notes

Simulation Details

MxN lattice (M = 400; N = 80)
periodic in x-direction
non-periodic in y-direction
3 main states (active, inactive, and refractory), but it can be split into more states 

Total activation effect factors in local and global interactions



In [1]:
import numpy as np

M = 400
N = 80

# Create a 2D lattice grid
lattice = np.zeros((M, N))

# Apply periodic boundary conditions on the x-axis
for i in range(M):
    lattice[i, 0] = lattice[i, N-1]
    lattice[i, N-1] = lattice[i, 0]

print(lattice)


[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
